# First go at brain activation

In [ ]:
# import common modules
import numpy as np  # the Python array package
import matplotlib.pyplot as plt  # the Python plotting package
# Display array values to 6 digits of precision
np.set_printoptions(precision=4, suppress=True)

# Image library.
import nibabel as nib

# Course library.
import nipraxis

## The simplest possible way to find task activation

We are going to look at a functional MRI task run, taken while the subject did
a task.  The task was a block design, where the subject rested, then thought of
as many verbs as they could for 30 seconds, then rested for 30 seconds, and so
on.

We are interested to find voxels that have higher signal for the scans while the subject was doing the task, then when the subject was resting.

One very simple way of doing this, is the following:

* Identify all the scans taken while the subject was doing the task.  Call
  these the *task volumes*.  Take an average across volume, of these task
  volumes to get a *task average* volume.
* Do the same thing for the rest volumes to get a *rest average* volume.
* Subtract the rest average volume from the task average volume to get the
  *difference average* volume.

Voxels with high signal in the *difference average* volume are voxels that had
higher average signal during the task scans than during the rest scans.  These
may be voxels *activated* by the task.

## The task time-course

We have previously had a look at the file `ds114_sub009_t2r1.nii`. This is a 4D
FMRI image.

In [ ]:
# Fetch the data file to this computer.
bold_fname = nipraxis.fetch_file('ds114_sub009_t2r1.nii')
# Show the filename
bold_fname

Now we want to see whether we can detect any signal in that image relating to
the task.

The task was a block design with 10 seconds rest followed by 7 repeats of (30
seconds when the subject thought of verbs followed by 30 seconds rest). This
is called a “covert” task, because the subjects were thinking of verbs instead
of saying them.

We next fetch the text file containing information about the block onsets. It
comes from subject 9 task 2 and run 1 of the [ds114
dataset](https://openfmri.org/dataset/ds000114), from the [OpenFMRI](https://openfmri.org/) project.

In [ ]:
# Fetch the task definition file to this computer.
stim_fname = nipraxis.fetch_file('ds114_sub009_t2r1_cond.txt')
# Show the filename
stim_fname

The file has one line of text per “on” block, giving the onset time of the
block (in seconds), the duration of the block (in seconds) and the amplitude
(expected amount of activation for this block - not used in this case).

Here are the first four lines:

```
10  30.000000   1
70  30.000000   1
130 30.000000   1
190 30.000000   1
```

Read the file into a (number of blocks by 3) array called `task`.

In [ ]:
#- Read the file into an array called "task".
#- "task" should have 3 columns (onset, duration, amplitude)
task = ...
# Show the result.
task

The repetition time (time to repeat, TR) for this FMRI run was 2.5 seconds. We
need to convert the onsets and durations to TRs - so for example the first
onset was at 10 seconds, which was the start of TR 4 (10 / 2.5).

Select out the first two columns of task, and divide by the TR to convert the
onset and duration times to be in terms of TRs instead of seconds:

In [ ]:
#- Select first two columns and divide by TR
ons_durs = ...
# Show the result
ons_durs

In [ ]:
assert ons_durs.shape == (7, 2)
assert np.all(ons_durs[-1] == [148, 12])

Our next step is to make an on-off vector that is 0 when the subject is doing
nothing and 1 when the subject is doing the covert verb task.

The vector will have one value (either 0 or 1) for each TR.

First use nibabel to load the image `ds114_sub009_t2r1.nii`. Check the image
shape to find the number of TRs.

In [ ]:
#- Load the image and check the image shape to get the number of TRs
img = ...
# Show the shape of the image.
img.shape

Next make a new vector called `time_course` with one entry per TR, with all
elements in the vector being zero:

In [ ]:
#- Make new zero vector
time_course = ...

In [ ]:
assert isinstance(time_course, np.ndarray)
assert time_course.shape == (img.shape[-1],)

Loop over the rows in the onsets / durations array to give you an onset /
duration pair. For each of these pairs, set the matching positions in the
`time_course` vector to 1. For example, the first pair will be `4, 12`. That
means the task started at the beginning of scan index 4, and lasted for 12
scans. There should be 12 consecutive 1 values in `time_course`,
starting at index 4. Index 4 + 12 = 16 should be zero again, because there are
12 values starting at (including) 4 going up to (but *not* including) 16.

In [ ]:
# Try running this if you don't believe me
len(range(4, 16))

So, for the first row, you will want to set `time_course[4]` through
`time_course[15]` equal to 1.

**Hint** - you may consider converting `ons_durs` to integers to make your task
easier.

In [ ]:
#- Fill in values of 1 for positions of on blocks in time course
...
...
# Show the result
time_course

Plot the time course:

In [ ]:
#- Plot the time course

## Comparing task to rest

Make a Boolean array `is_task_tr` which is True when `time_course` is 1
and False otherwise.

Make another array `is_rest_tr` that is the opposite - True when
`time_course` is 0 and False otherwise.

In [ ]:
#- Make two Boolean arrays encoding task, rest
is_task_tr = ...
is_rest_tr = ...

In [ ]:
assert np.count_nonzero(is_task_tr) == 84  # 84 task scans
assert np.count_nonzero(is_rest_tr) == 89  # 89 rest scans

Read the image data as a 4D array:

In [ ]:
#- Read the image data  an array.
data = ...
# Show the shape of the resulting array.
data.shape

Remember that the 4D array consists of one volume (3D array) per TR.

We want to select the volumes where the time course is 1 (task volumes).  Do
this by slicing, using the Boolean array you just made.

In [ ]:
#- Create a new 4D array only containing the task volumes
on_volumes = ...

In [ ]:
assert on_volumes.shape == data.shape[:-1] + (84,)

Select the volumes where the time course is 0 (rest volumes):

In [ ]:
#- Create a new 4D array only containing the rest volumes
off_volumes = ...

In [ ]:
assert off_volumes.shape == data.shape[:-1] + (89,)

We want to know whether there is a difference in signal in the task volumes
compared to the rest volumes. Take the mean over the task volumes and mean
over the rest volumes. You should end up with two 3D volumes.

In [ ]:
#- Create the mean volume across all the task volumes.
#- Then create the mean volume across all the rest volumes
on_mean = ...
off_mean = ...

In [ ]:
assert on_mean.shape == data.shape[:-1]
assert off_mean.shape == data.shape[:-1]

Now subtract the rest mean from the task mean to get a difference volume.

In [ ]:
#- Create a difference volume
difference = ...

In [ ]:
assert difference.shape == data.shape[:-1]

Show a slice over the third dimension of the difference volume, from somewhere
around the center of the third axis:

In [ ]:
#- Show a slice over the third dimension

This is the difference between activation and rest. It looks a little strange.
Maybe there are some artefacts here.

## Fixing the artefact

One way of looking for artefacts in a 4D image to find volumes with unusually
high variance / standard deviation.

There is one volume in this 4D image with particularly high standard deviation.

In fact, this volume has a particularly high standard deviation across voxels.

The standard deviation across voxels is the standard deviation of all the voxels in the 3D volume.  Thus, there will be one standard deviation across voxels, per volume.

Calculate the standard deviation across voxels for each volume.

In [ ]:
#- Calculate the SD across voxels for each volume
#- Identify the outlier volume
stds = ...
# Plot the result
plt.plot(stds)

Which volume is this (what position)?

Use your slicing skills to remove this volume from your selection of rest (off)
volumes.

In [ ]:
#- Use slicing to remove outlier volume from rest volumes
off_volumes_fixed = ...
# Show the shape
off_volumes_fixed.shape

In [ ]:
assert off_volumes_fixed.shape == data.shape[:-1] + (88,)

Make a new mean for the rest volumes, and subtract this mean from the mean for
the task volumes to make a new difference image.

Give the new difference image a new name, so we can compare to the old
difference image later.

In [ ]:
#- Make new mean for rest volumes, subtract from task mean
difference_fixed = ...

In [ ]:
assert difference_fixed.shape == data.shape[:-1]

Show an example slice from the new difference volume. Show the same slice from
the old difference volume, using matplotlib.

In [ ]:
#- show slice from old difference volume
plt.imshow(...)

In [ ]:
#- show same slice from new difference volume
plt.imshow(...)